In [1]:
# Import dependencies
from time import sleep
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [C:\Users\Boss\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


## Extract: scrape recipe data from bbc.co.uk/food

In [3]:
url = "https://www.bbc.co.uk/food/cuisines"

# Use splinter to navigate the site
browser.visit(url)

In [4]:
# Create BeautifulSoup object; parse with 'lxml'
html = browser.html
soup = bs(html, 'lxml')

In [5]:
# Retrieve all text with links to each cuisine
cuisines = [cuisine.get_text() for cuisine in soup.find_all("h3", class_ = 'promo__title gel-pica')]
cuisines

['African',
 'American',
 'British',
 'Caribbean',
 'Chinese',
 'East European',
 'French',
 'Greek',
 'Indian',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Mexican',
 'Nordic',
 'North African',
 'Pakistani',
 'Portuguese',
 'South American',
 'Spanish',
 'Thai and South-East Asian',
 'Turkish and Middle Eastern']

In [6]:
# Define a function to add scraped data to csv files
from csv import writer

def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='', encoding='utf-16') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [7]:
for cuisine in cuisines:
    
    # Click the links to all recipes of each cuisine.
    browser.links.find_by_partial_text(cuisine).click()
    sleep(1)
    browser.links.find_by_partial_text(f"all {cuisine} recipes").click()
    sleep(1)
    html = browser.html
    soup = bs(html, 'lxml')
    
    # Retrieve all recipe href links to each recipe in the current page
    recipe_hrefs = [recipe['href'] for recipe in soup.find_all("a", class_ = "promo")]
    
    for recipe_href in recipe_hrefs:
        browser.links.find_by_href(recipe_href).click()
        current_url = browser.url
        sleep(2)
        html = browser.html
        soup = bs(html, 'lxml')

        # Retrieve recipe name
        recipe = soup.find("h1").get_text()
        
        # Retrieve list of ingredients
        ingredients = [ingredient.get_text() for ingredient in soup.find_all(
            "a", class_ = "recipe-ingredients__link")]
        full_ingredients = [ingredient.get_text() for ingredient in soup.find_all(
            "li", class_ = "recipe-ingredients__list-item")]
        
        # Retrieve recipe image if available
        try:
            recipe_image = soup.select_one("div.recipe-media__image img")["src"]
        except Exception as e:
            append_list_as_row("errors.csv", [e, current_url])
            recipe_image = None
            
        # Append the list of scraped data to a csv file
        append_list_as_row("cuisine_ingredients.csv", [cuisine, recipe, ingredients, full_ingredients, recipe_image])
            
        browser.back()
        sleep(2)
        
    html = browser.html
    soup = bs(html, 'lxml')
    
    # Click the next recipe page in navigation bar at the bottom of the page (if available)
    try:
        max_page = int(soup.find_all("li", class_="pagination__list-item")[-2].get_text())

        for i in range(2, max_page+1, 1):
            browser.links.find_by_partial_href(f"page={i}").click()

            sleep(2)
            html = browser.html
            soup = bs(html, 'lxml')

            # Retrieve all recipe href links to each recipe in the current page
            recipe_hrefs = [recipe['href'] for recipe in soup.find_all("a", class_ = "promo")]

            for recipe_href in recipe_hrefs:
                browser.links.find_by_href(recipe_href).click()
                current_url = browser.url
                sleep(2)
                html = browser.html
                soup = bs(html, 'lxml')

                # Retrieve recipe name
                recipe = soup.find("h1").get_text()

                # Retrieve list of ingredients
                ingredients = [ingredient.get_text() for ingredient in soup.find_all(
                    "a", class_ = "recipe-ingredients__link")]
                full_ingredients = [ingredient.get_text() for ingredient in soup.find_all(
                    "li", class_ = "recipe-ingredients__list-item")]

                # Retrieve recipe image if available
                try:
                    recipe_image = soup.select_one("div.recipe-media__image img")["src"]
                except Exception as e:
                    append_list_as_row("errors.csv", [e, current_url])
                    recipe_image = None
            
                # Append the list of scraped data to a csv file
                append_list_as_row("cuisine_ingredients.csv", [cuisine, recipe, ingredients, full_ingredients, recipe_image])
                
                browser.back()
                sleep(2)

        # Back to the starting url
        browser.visit(url)
        sleep(2)

    except IndexError:
        # Back to the starting url
        browser.visit(url)
        sleep(2)